1. Obtain the data that is in the table of postal codes and transform the data into a pandas dataframe

In [3]:
import pandas as pd
import numpy as np

#Read the table
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
#Drop the rows with the text "Not assigned" in column "Borough"
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)


Note: It is no necesary to combine the rows, because the table contains the Neighborhood grouped by Postal Code 

In [4]:
#Replace NaN Values in Neighborhood by Borough Cell
df.Neighborhood.fillna(df.Borough, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [ ]:
df.shape

In [5]:
!pip install geocoder

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Looking in indexes: http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/release/+simple/, http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/dev/+simple/, https://pypi.python.org/simple/
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491062 sha256=2c53dcdda2040169e6dce94b3a76b7ce2b306a1902ecfe0f1059d2d62b06bf5e
  Stored in directory: C:\Users\lmgonzalez\AppData\Local\pip\Cache\wheels\ea\11\1c\94a80a1a23e82687271425d42915f9ade4965d0fdcf327ddb1
Successfully built future


In [6]:
import geocoder
lat=[]
long=[]

for i in range(df['Postal Code'].count()):
    lat_lng_coords=None
    while(lat_lng_coords is None):
      g=geocoder.arcgis('{}, Toronto, Canada'.format(df.iloc[i,0]))
      lat_lng_coords=g.latlng
      lat.append(lat_lng_coords[0])
      long.append(lat_lng_coords[1])

df["Latitud"]=lat
df["longitude"]=long

df

,Postal Code,Borough,Neighborhood,Latitud,longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


3. Explore and cluster the neighborhoods in Toronto

In [15]:
!pip install geopy
!pip install folium
import folium
from geopy.geocoders import Nominatim
#Create map of Toronto

address = 'Toronto, CA'
geolocator= Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude = location.longitude
map_toronto =folium.Map(location=[latitude, longitude], zoom_start=10)

# As was suggested in the exercise, I 'll work with Borough that contains the word "Toronto"

Toronto_data=df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)

#add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitud'], Toronto_data['longitude'], Toronto_data['Borough'],Toronto_data['Neighborhood']):
    label ='{},{}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Looking in indexes: http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/release/+simple/, http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/dev/+simple/, https://pypi.python.org/simple/

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Looking in indexes: http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/release/+simple/, http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/dev/+simple/, https://pypi.python.org/simple/


In [126]:
Toronto_data.head()
#Toronto_data.shape

,Postal Code,Borough,Neighborhood,Latitud,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


Now lets apply clustering

In [27]:
#Set the Credentials in Foursquare

CLIENT_ID = 'MZOOMFFAPZXYBQD0JSHQOOV5TFTEUW3VDYZEB43OQW3L1QPF' # your Foursquare ID
CLIENT_SECRET = '1M4HBKDIZIGHFUUCJDLXB31F3IEKMU21HU1M5YCX3YPFSH1T' # your Foursquare Secret
VERSION = '20200502' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MZOOMFFAPZXYBQD0JSHQOOV5TFTEUW3VDYZEB43OQW3L1QPF
CLIENT_SECRET:1M4HBKDIZIGHFUUCJDLXB31F3IEKMU21HU1M5YCX3YPFSH1T


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


Lest do the steps to explore Neigborhoods in Toronto

In [38]:
#Crate function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# Create a Data Frame for Toronto Venues

toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitud'],
                                   longitudes=Toronto_data['longitude']
                                  )



Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [41]:
# Chaeck the size of the resulting frame

print (toronto_venues.shape)
toronto_venues.head()

(1581, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [42]:
# See how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,64,64,64,64,64,64
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",64,64,64,64,64,64
Central Bay Street,54,54,54,54,54,54
Christie,12,12,12,12,12,12
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [109]:
# Encoding 

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name="Neighborhood"
first_col=toronto_onehot.pop(col_name)

toronto_onehot.insert(0, col_name, first_col)
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
toronto_onehot.shape

(1581, 227)

Group rows by Neighborhood and by taking the mean of the frequency of ocurrence of each category

In [112]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape



(39, 225)

See the top venues

In [118]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2         Cheese Shop  0.03
3  Seafood Restaurant  0.03
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.07
2  Thrift / Vintage Store  0.05
3               Gift Shop  0.05
4                   Diner  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2           Restaurant  0.04
3  Japanese Restaurant  0.03
4     Asian Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0               Café  0.06
1        Coffee Shop  0.06
2  French Restaurant  0.05
3         Restaurant  0.05
4               Park  0.05


----Central Bay

In [119]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Seafood Restaurant,Cheese Shop,Hotel,Restaurant,Breakfast Spot,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Diner,Gift Shop,Restaurant,Thrift / Vintage Store,North Indian Restaurant,Brewery,Caribbean Restaurant,Chiropractor
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Asian Restaurant,Japanese Restaurant,Sandwich Place,Thai Restaurant,Salon / Barbershop,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,French Restaurant,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Lounge,Bar,Speakeasy
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Sandwich Place,Clothing Store,Plaza,Bubble Tea Shop,Pizza Place,Bookstore


In [116]:
#set number of clusters
from sklearn.cluster import KMeans

kclusters=5
Clustering =toronto_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Clustering)
kmeans.labels_[:]



array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 1, 1, 1, 4, 1, 1, 0,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1])

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitud,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1,Pub,Athletics & Sports,Café,Food Truck,Distribution Center,Furniture / Home Store,Coffee Shop,Antique Shop,French Restaurant,Tech Startup
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Café,Yoga Studio,Sandwich Place,Sushi Restaurant,Bookstore,Fried Chicken Joint,Burrito Place,Smoothie Shop,Middle Eastern Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Cosmetics Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Clothing Store,Lingerie Store
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,1,Church,Health Food Store,Pub,Trail,Yoga Studio,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


Plot clusters

In [132]:
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitud'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Looking in indexes: http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/release/+simple/, http://usmdckdap10367.nix.us.kworld.kpmg.com/ignite/dev/+simple/, https://pypi.python.org/simple/


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


There are 5  clusters: Lets examine cluster 1, it seems to be associated with parks or places for playground

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Toronto,0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Food,Flower Shop,Fish Market
21,Central Toronto,0,Gym / Fitness Center,Park,Yoga Studio,Donut Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
23,Central Toronto,0,Playground,Gym Pool,Garden,Park,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop
33,Downtown Toronto,0,Playground,Candy Store,Park,Grocery Store,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop


Cluster 2 - it seems to be associated with cofees shops and restaurants

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Pub,Athletics & Sports,Café,Food Truck,Distribution Center,Furniture / Home Store,Coffee Shop,Antique Shop,French Restaurant,Tech Startup
1,Downtown Toronto,1,Coffee Shop,Café,Yoga Studio,Sandwich Place,Sushi Restaurant,Bookstore,Fried Chicken Joint,Burrito Place,Smoothie Shop,Middle Eastern Restaurant
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Cosmetics Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar
3,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Clothing Store,Lingerie Store
4,East Toronto,1,Church,Health Food Store,Pub,Trail,Yoga Studio,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Café,Bakery,Seafood Restaurant,Cheese Shop,Hotel,Restaurant,Breakfast Spot,Beer Bar
6,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Sandwich Place,Clothing Store,Plaza,Bubble Tea Shop,Pizza Place,Bookstore
7,Downtown Toronto,1,Grocery Store,Café,Athletics & Sports,Park,Baby Store,Coffee Shop,Candy Store,Playground,Fish Market,Fish & Chips Shop
8,Downtown Toronto,1,Coffee Shop,Café,Clothing Store,Restaurant,Salad Place,Sushi Restaurant,Thai Restaurant,Gym,Hotel,Deli / Bodega
9,West Toronto,1,Park,Pharmacy,Athletics & Sports,Brazilian Restaurant,Café,Furniture / Home Store,Bank,Bakery,Gym,Grocery Store


Lets examine cluster 3, only one place belongs to this group. (home service and womens store)

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Home Service,Women's Store,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Lets examine cluster 4, only one place belongs to this group. (parks)

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Theme Park,Harbor / Marina,Farm,Park,Donut Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


last cluster

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Bus Line,Swim School,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
